In [63]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
import itertools
import os

In [75]:
# 設定全域參數
K_VALUE = 6              # 保持 K=6 捕捉高階特徵
MAX_SEQ_LEN = 750        # 提升長度
BATCH_SIZE = 64          # 提升 Batch Size 穩定梯度
EPOCHS = 1
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [65]:
# --- 步驟 1: 資料讀取與權重優化 ---
def load_and_preprocess():
    train_df = pd.read_csv('dataset/train.csv')
    val_df = pd.read_csv('dataset/validation.csv')
    test_df = pd.read_csv('dataset/test.csv')

    def clean_seq(seq): return seq.strip('<>').upper()
    for df in [train_df, val_df, test_df]:
        df['seq_clean'] = df['NucleotideSequence'].apply(clean_seq)

    le = LabelEncoder()
    train_df['label'] = le.fit_transform(train_df['GeneType'])
    val_df['label'] = le.transform(val_df['GeneType'])
    test_df['label'] = le.transform(test_df['GeneType'])

    # 權重修正：使用平方根倒數平滑化，防止過度補償小類別
    counts = train_df['label'].value_counts().sort_index().values
    weights = torch.FloatTensor(1.0 / np.sqrt(counts + 1)).to(DEVICE)

    # 驗證 1: 標籤映射與平滑權重計算成功。
    print(f"✅ 資料載入完成。類別權重已平滑處理。")
    return train_df, val_df, test_df, le, weights

In [66]:
# --- 步驟 2: 特徵工程 (K-mer Vectorizer) ---
def get_vectorizer():
    bases = ['A', 'C', 'G', 'T']
    vocab = [''.join(p) for p in itertools.product(bases, repeat=K_VALUE)]
    return CountVectorizer(vocabulary=vocab)

class GenomicDataset(Dataset):
    def __init__(self, df, cv, max_len=750):
        self.labels = df['label'].values
        self.seqs = df['seq_clean'].values
        self.max_len = max_len

        # K-mer 提取
        kmers_list = [' '.join([s[i:i+K_VALUE] for i in range(len(s)-K_VALUE+1)]) for s in self.seqs]
        self.kmer_features = cv.transform(kmers_list).toarray().astype(np.float32)
        self.char_map = {'A': 1, 'C': 2, 'G': 3, 'T': 4}

    def __len__(self): return len(self.labels)

    def __getitem__(self, idx):
        seq = self.seqs[idx]
        encoded = [self.char_map.get(b, 0) for b in seq[:self.max_len]]
        padded = encoded + [0] * (self.max_len - len(encoded))
        return {
            'seq': torch.LongTensor(padded),
            'kmer': torch.FloatTensor(self.kmer_features[idx]),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [67]:
# --- 步驟 3: 多尺度 CNN + MLP 模型 ---
class AdvancedDNAHybridModel(nn.Module):
    def __init__(self, num_classes, kmer_dim):
        super().__init__()
        self.embedding = nn.Embedding(5, 64, padding_idx=0)

        # 多尺度卷積分支
        self.conv3 = nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.conv7 = nn.Conv1d(64, 64, kernel_size=7, padding=3)
        self.conv15 = nn.Conv1d(64, 64, kernel_size=15, padding=7)

        self.bn_cnn = nn.BatchNorm1d(192) # 64 * 3
        self.pool = nn.AdaptiveAvgPool1d(1)

        # K-mer 分支 (強化層)
        self.kmer_mlp = nn.Sequential(
            nn.Linear(kmer_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4)
        )

        self.classifier = nn.Sequential(
            nn.Linear(192 + 512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, seq, kmer):
        x_seq = self.embedding(seq).transpose(1, 2)

        # 提取不同尺度的局部特徵
        c3 = torch.relu(self.conv3(x_seq))
        c7 = torch.relu(self.conv7(x_seq))
        c15 = torch.relu(self.conv15(x_seq))

        cnn_feat = torch.cat([c3, c7, c15], dim=1)
        cnn_feat = self.pool(self.bn_cnn(cnn_feat)).squeeze(-1)

        x_kmer = self.kmer_mlp(kmer)

        combined = torch.cat((cnn_feat, x_kmer), dim=1)
        return self.classifier(combined)

In [73]:
# --- 步驟 4: 訓練與評估流程 ---
def run_pipeline():
    train_df, val_df, test_df, le, class_weights = load_and_preprocess()
    cv = get_vectorizer()

    train_ds = GenomicDataset(train_df, cv, MAX_SEQ_LEN)
    val_ds = GenomicDataset(val_df, cv, MAX_SEQ_LEN)
    test_ds = GenomicDataset(test_df, cv, MAX_SEQ_LEN)

    # 修改重點：加入 drop_last=True
    train_loader = DataLoader(
        train_ds,
        batch_size=BATCH_SIZE,
        shuffle=True,
        drop_last=True  # 防止最後一個 batch 只有 1 個樣本導致 BatchNorm 崩潰
    )

    # 驗證集與測試集通常不需 drop_last，因為我們會呼叫 model.eval()
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

    model = AdvancedDNAHybridModel(len(le.classes_), 4**K_VALUE).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    # 驗證 2: 模型結構與維度檢查通過。開始訓練...
    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            out = model(batch['seq'].to(DEVICE), batch['kmer'].to(DEVICE))
            loss = criterion(out, batch['label'].to(DEVICE))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                out = model(batch['seq'].to(DEVICE), batch['kmer'].to(DEVICE))
                val_loss += criterion(out, batch['label'].to(DEVICE)).item()

        scheduler.step(val_loss)
        print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss/len(train_loader):.4f} | Val Loss: {val_loss/len(val_loader):.4f}")

    # 驗證 3: 訓練完成。正在計算測試集指標...
    model.eval()
    all_preds = []
    with torch.no_grad():
        for batch in test_loader:
            logits = model(batch['seq'].to(DEVICE), batch['kmer'].to(DEVICE))
            all_preds.extend(torch.argmax(logits, dim=1).cpu().numpy())

    # 輸出最終報表
    print(f"\n🚀 提升後的測試集準確率: {accuracy_score(test_df['label'], all_preds):.4f}")
    print(classification_report(test_df['label'], all_preds, target_names=le.classes_))

    # 驗證 4: 成功產出 results.csv。
    results = pd.DataFrame({'id': test_df['NCBIGeneID'], 'label': le.inverse_transform(all_preds)})
    results.to_csv('results.csv', index=False)
    torch.save(model, 'full_model_dna.pt')

In [76]:
if __name__ == "__main__":
    run_pipeline()

✅ 資料載入完成。類別權重已平滑處理。
Epoch 1/1 | Train Loss: 1.4593 | Val Loss: 1.1974

🚀 提升後的測試集準確率: 0.6713
                   precision    recall  f1-score   support

BIOLOGICAL_REGION       0.80      0.85      0.82      2651
            OTHER       0.00      0.00      0.00       133
   PROTEIN_CODING       0.20      0.57      0.29       184
           PSEUDO       0.79      0.65      0.71      3800
            ncRNA       0.38      0.64      0.48       894
             rRNA       0.76      0.90      0.83        72
            scRNA       0.00      0.00      0.00         1
            snRNA       0.00      0.00      0.00        38
           snoRNA       0.46      0.31      0.37       405
             tRNA       0.00      0.00      0.00       148

         accuracy                           0.67      8326
        macro avg       0.34      0.39      0.35      8326
     weighted avg       0.69      0.67      0.67      8326



C:\Users\jay07\miniconda3\envs\ML_final\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\jay07\miniconda3\envs\ML_final\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\jay07\miniconda3\envs\ML_final\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", 